In [31]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(".."))
from core.viz import plot_line, plot_bar, plot_heatmap, plot_corr_triangle, plot_cdf, plot_statistical_strip, plot_scatter
from core.s3 import S3AssetManager

In [32]:
notebook_name = "el_dorado_rendering"
s3 = S3AssetManager(notebook_name=notebook_name)

In [33]:
microbiologia = s3.read_csv("raw/dorado/rendering_microbiologia/microbiologia_el_dorado.csv")
evi = s3.read_csv("raw/dorado/rendering_microbiologia/evicerado_el_dorado.csv")
sensores = s3.read_csv("raw/dorado/rendering_microbiologia/sensores_el_dorado.csv")


microbiologia["date"] = pd.to_datetime(microbiologia["date"])
microbiologia = microbiologia[microbiologia["date"]<='2025-12-31'].copy()
microbiologia["month"] = microbiologia["date"].dt.to_period("M")
m = microbiologia["month"].dt.to_timestamp()
microbiologia ["month"] = m.dt.strftime("%b-%Y")
cats = pd.date_range(m.min(), m.max(), freq="MS").strftime("%b-%Y")
microbiologia["month"] = pd.Categorical(microbiologia["month"], categories=cats, ordered=True)


evi["fecha"] = pd.to_datetime(evi["fecha"])
evi["month"] = evi["fecha"].dt.to_period("M")
m = evi["month"].dt.to_timestamp()
evi ["month"] = m.dt.strftime("%b-%Y")
cats = pd.date_range(m.min(), m.max(), freq="MS").strftime("%b-%Y")
evi["month"] = pd.Categorical(evi["month"], categories=cats, ordered=True)

In [34]:
microbiologia["date"].max()

Timestamp('2025-12-31 00:00:00')

In [35]:

micro_month = microbiologia.groupby(["month", "etapa", "microorganismo"]).agg(
    prev=("have_micro", "mean"),
    result=('result', 'mean'),
    ph=('ph', 'mean'),
    orp=('orp', 'mean'),
    cloro=('cloro', 'mean'),
    n_analysis=('have_micro', 'count')
    ).reset_index()
micro_month["prev"] = micro_month["prev"] * 100


evi_group = evi.groupby("month").agg(
                    pollos_remisionados=('pollos_remisionados', "sum"),
                    buches_eviceracion=('buches_zona_eviceración', "sum"),
                    intestinos_eviceracion=('intestinos_zona_eviceración', "sum"),
                    gr_material_fecal=('gr_materia_fecal', "sum"),
           ).reset_index()
evi_group["kg_descuento"] = (
evi_group["buches_eviceracion"]
+ evi_group["intestinos_eviceracion"])

evi_group

df_kilos = evi_group[["month", "kg_descuento"]].copy()
df_kilos["etapa"] = "Descuento (Kg)"
df_kilos = df_kilos.rename(columns={"kg_descuento": "prev"})
df_kilos

micro_month_salmonella = micro_month[micro_month["microorganismo"] == "Salmonella"]
micro_month_campy = micro_month[micro_month["microorganismo"] == "Campylobacter"]
df_salmo = pd.concat([micro_month_salmonella, df_kilos], ignore_index=True)
df_campy = pd.concat([micro_month_campy, df_kilos], ignore_index=True)

/var/folders/1g/77kw2x4j5678s_87_sqc1fpc0000gp/T/ipykernel_9696/1250481969.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/var/folders/1g/77kw2x4j5678s_87_sqc1fpc0000gp/T/ipykernel_9696/1250481969.py:12: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [36]:

fig = plot_line(
    df=df_salmo,
    x_col="month",
    y_col="prev",
    group_col="etapa",
    secondary_y_col="Descuento (Kg)",
    secondary_y_title="Kilogramos",
    order_x=['Jan-2025', 'Feb-2025', 'Mar-2025', 'Apr-2025', 'May-2025',
       'Jun-2025', 'Jul-2025', 'Aug-2025', 'Sep-2025', 'Oct-2025',
       'Nov-2025', 'Dec-2025', 'Jan-2026'],
    x_title="Mes",
    y_title="Prevalencia (%)",
    title="Prevalencia Salmonella vs Kilos de Descuento",
    line_colors={
        "Poll Ext Visceras": "#1f4e5f",
        "ciegos": "#3d8b7a",
        "salida chiller": "#9db494",
        "prechiller": "#f2b47e",
        "Descuento (Kg)": "#ef8a82"
    },
    height = 500,
    width = 1100,
)

fig.show()
s3.save_plotly_html(fig, "salmonella_line.html")

In [38]:

fig = plot_line(
    df=df_campy,
    x_col="month",
    y_col="prev",
    group_col="etapa",
    secondary_y_col="Descuento (Kg)",  # <--- Esto activa el eje derecho
    secondary_y_title="Kilogramos",
    x_title="Mes",
    y_title="Prevalencia (%)",
    order_x=['Jan-2025', 'Feb-2025', 'Mar-2025', 'Apr-2025', 'May-2025',
       'Jun-2025', 'Jul-2025', 'Aug-2025', 'Sep-2025', 'Oct-2025',
       'Nov-2025', 'Dec-2025', 'Jan-2026'],
    title="Prevalencia Campylobacter vs Kilos de Descuento",
    line_colors={
        "Poll Ext Visceras": "#1f4e5f",
        "ciegos": "#3d8b7a",
        "salida chiller": "#9db494",
        "prechiller": "#f2b47e",
        "Descuento (Kg)": "#ef8a82" # Color salmón para el eje derecho
    },
    height = 500,
   width = 1100,
)

fig.show()
s3.save_plotly_html(fig, "campy_line.html")

In [39]:
micro_month

,month,etapa,microorganismo,prev,result,ph,orp,cloro,n_analysis
0,Jan-2025,Poll Ext Visceras,Campylobacter,46.666667,4130.600000,NaN,NaN,NaN,15
1,Jan-2025,Poll Ext Visceras,Salmonella,46.666667,607.933333,NaN,NaN,NaN,15
2,Jan-2025,ciegos,Campylobacter,56.410256,137270.846154,NaN,NaN,NaN,39
3,Jan-2025,ciegos,Salmonella,42.500000,9445.925000,NaN,NaN,NaN,40
4,Jan-2025,prechiller,Campylobacter,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...
91,Dec-2025,ciegos,Salmonella,12.820513,0.358974,NaN,NaN,NaN,39
92,Dec-2025,prechiller,Campylobacter,47.368421,117.157895,NaN,NaN,NaN,19
93,Dec-2025,prechiller,Salmonella,5.000000,0.100000,NaN,NaN,NaN,20
94,Dec-2025,salida chiller,Campylobacter,48.000000,6.800000,5.857600,624.840000,4.360000,25


In [40]:
last_month=micro_month["month"].max()
fig =plot_heatmap(
    df=micro_month[micro_month["month"] == last_month],
    x_col="etapa",
    y_col="microorganismo",
    value_col="prev",
    secondary_col="n_analysis",
    secondary_aggfunc="sum",
    aggfunc="mean",
    x_order= ["ciegos",  "prechiller", "salida chiller"], #"Poll Ext Visceras",
    value_unit="%",
    decimals_value=1,
    unit_position="suffix",
    secondary_prefix="",
    #colorscale="Blues",
    title=f"<b>Prevalencias y número de análisis en {last_month}</b>",
    show_secondary_labels=True,
    secondary_position="bottom",
    font_color="black",
    center_font_size=14,
    secondary_font_size=10,
    width=800,
    height=300,
    transparent_bg=True,
)
fig.show()
s3.save_plotly_html(fig, "prevalencia_heatmap.html")

In [41]:

for microorganismo in microbiologia["microorganismo"].unique():
    cond_microorganismo = microbiologia["microorganismo"] == microorganismo
    cond_last_month = microbiologia["month"] == microbiologia["month"].max()

    last_month_microorganismo = microbiologia[cond_microorganismo & cond_last_month].groupby(["etapa", "granja"]).agg(
    prev=("have_micro", "mean"),
    result=('result', 'mean'),
    ph=('ph', 'mean'),
    orp=('orp', 'mean'),
    cloro=('cloro', 'mean'),
    n_analysis=('microorganismo', 'count')
    ).reset_index()
    last_month_microorganismo["prev"] = last_month_microorganismo["prev"] * 100

    f = plot_heatmap(
        df=last_month_microorganismo,
        x_col="etapa",
        y_col="granja",
        value_col="prev",
        #secondary_col="n_analysis",
        secondary_aggfunc="sum",
        decimals_value=1,
        aggfunc="mean",
        x_order= ["ciegos",  "prechiller", "salida chiller"], #"Poll Ext Visceras",
        value_unit="%",
        unit_position="suffix",
        secondary_prefix="",
        #colorscale="Blues",
        title=f"<b>Prevalencia de {microorganismo} en {last_month} por granja</b>",
        show_secondary_labels=True,
        secondary_position="bottom",
        font_color="black",
        center_font_size=14,
        secondary_font_size=10,
        width=800,
        height=700,
        transparent_bg=True,
    )
    f.show()
    name = f"prev_last_month_{microorganismo}.html"
    print(name)
    s3.save_plotly_html(f, name)


prev_last_month_Salmonella.html


prev_last_month_Campylobacter.html


In [42]:
cond_chiller = microbiologia["etapa"] == "salida chiller"
cond_last_month = microbiologia["month"] == microbiologia["month"].max()
chiller_last_month = microbiologia[cond_chiller & cond_last_month]

index_cols = ['ph', 'orp', 'cloro']

df_pivot = chiller_last_month.pivot_table(
    index=index_cols,
    columns='microorganismo',
    values=['log_result', 'have_micro'],
    aggfunc='mean'
)

df_pivot.columns = [f'{val}_{micro.lower()}' for val, micro in df_pivot.columns]

df_final = df_pivot.reset_index()
df_final

,ph,orp,cloro,have_micro_campylobacter,have_micro_salmonella,log_result_campylobacter,log_result_salmonella
0,5.30,493.0,4.5,NaN,0.0,NaN,0.0
1,5.77,465.0,4.6,1.0,0.0,0.301030,0.0
2,5.79,457.0,4.0,1.0,0.0,0.698970,0.0
3,5.79,469.0,4.7,NaN,0.0,NaN,0.0
4,5.80,431.0,4.1,NaN,0.0,NaN,0.0
...,...,...,...,...,...,...,...
66,5.94,438.0,4.6,1.0,0.0,0.477121,0.0
67,5.94,453.0,4.5,NaN,0.0,NaN,0.0
68,5.95,451.0,4.4,NaN,0.0,NaN,0.0
69,5.96,483.0,4.3,NaN,0.0,NaN,0.0


In [43]:
fig = plot_corr_triangle(
    df_final,value_cols=["ph", "have_micro_campylobacter","cloro", "orp", "have_micro_salmonella"], method='pearson', 
    title='<b>Correlación (Pearson) — triángulo superior</b>',
     decimals=2, width=1000, height=400, 
     )
fig.show()
s3.save_plotly_html(fig, "correlacion_chiller.html")

In [44]:
result_group = microbiologia[microbiologia['log_result']>0].groupby(['microorganismo', 'etapa', 'month']).agg(
    avg_log_result=('log_result', 'mean'),
    sigma_log_result=('log_result', 'std'),
    cv_log_result=('log_result', lambda x: x.std() / x.mean()*100)
).reset_index()
result_group

/var/folders/1g/77kw2x4j5678s_87_sqc1fpc0000gp/T/ipykernel_9696/401795164.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,microorganismo,etapa,month,avg_log_result,sigma_log_result,cv_log_result
0,Campylobacter,Poll Ext Visceras,Jan-2025,2.531813,1.701154,67.191155
1,Campylobacter,Poll Ext Visceras,Feb-2025,4.498117,2.722326,60.521453
2,Campylobacter,Poll Ext Visceras,Mar-2025,1.562397,0.928925,59.455130
3,Campylobacter,Poll Ext Visceras,Apr-2025,2.129170,1.133605,53.241639
4,Campylobacter,Poll Ext Visceras,May-2025,1.611425,1.595176,98.991646
...,...,...,...,...,...,...
91,Salmonella,salida chiller,Aug-2025,2.146128,NaN,NaN
92,Salmonella,salida chiller,Sep-2025,0.965533,0.875397,90.664634
93,Salmonella,salida chiller,Oct-2025,2.184673,2.838421,129.924277
94,Salmonella,salida chiller,Nov-2025,0.301030,NaN,NaN


In [45]:
plot_line(
    df=result_group[result_group['etapa']=='salida chiller'],
    x_col='month',
    y_col='cv_log_result',
    group_col='microorganismo',
    order_x=['Jan-2025', 'Feb-2025', 'Mar-2025', 'Apr-2025', 'May-2025',
       'Jun-2025', 'Jul-2025', 'Aug-2025', 'Sep-2025', 'Oct-2025',
       'Nov-2025', 'Dec-2025', 'Jan-2026'],
    x_title='Mes',
    y_title='CV log',
    title='CV log por microorganismo cuando se detecta prevalencia'
)

In [46]:
plot_line(
    df=result_group[result_group['etapa']=='salida chiller'],
    x_col='month',
    y_col='avg_log_result',
    group_col='microorganismo',
    order_x=['Jan-2025', 'Feb-2025', 'Mar-2025', 'Apr-2025', 'May-2025',
       'Jun-2025', 'Jul-2025', 'Aug-2025', 'Sep-2025', 'Oct-2025',
       'Nov-2025', 'Dec-2025', 'Jan-2026'],
    x_title='Mes',
    y_title='Promedio log',
    title='Promedio log por microorganismo cuando se detecta prevalencia'
)

In [47]:
fig = plot_cdf(chiller_last_month, value_col='ph', group_col='month', 
width=800, height=500)
s3.save_plotly_html(fig, "cdf_ph.html")


In [48]:

fig = plot_cdf(chiller_last_month, value_col='orp', group_col='month', width=800, height=500)
s3.save_plotly_html(fig, "cdf_orp.html")

In [49]:
fig = plot_cdf(chiller_last_month, value_col='cloro', group_col='month', width=800, height=500)
s3.save_plotly_html(fig, "cdf_cloro.html")

In [50]:
### Analisis de los sensores

In [51]:
cond_min_cloro = sensores["cloro_chiller"] < 2
cond_max_cloro = sensores["cloro_chiller"] > 5
sensores.loc[(cond_min_cloro | cond_max_cloro), "cloro_chiller"] = sensores["cloro_chiller"].median()

In [53]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.colors as mcolors
import skfda
from skfda.exploratory.visualization import Boxplot
from skfda.exploratory.depth import ModifiedBandDepth

# ==============================================================================
# 1. CONFIGURACIÓN GENERAL (Fácil de editar)
# ==============================================================================
CONFIG = {
    # Columnas disponibles: 'cloro_chiller', 'ph_chiller', 'orp_chiller'
    'VARIABLE_OBJETIVO': 'cloro_chiller',  
    'FRECUENCIA_MUESTREO': '1h',           # Ej: '15min', '30min', '1h'
    
    # Filtro de meses
    'MESES': [10, 11, 12],
    'NOMBRES_MESES': {10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'},
    
    # Limpieza de datos (Valores fuera de rango se reemplazan por la mediana)
    # Si no quieres filtrar, pon None en los valores.
    'LIMITES_LIMPIEZA': {
        'min': 2.0, 
        'max': 5.0
    },

    # Estética
    'COLORES': {
        'mediana': "#000000",       # Negro
        'banda': "#778899",         # Gris Azulado
        'marco': "#202020",
        'grid': "#E0E0E0"
    }
}

# ==============================================================================
# 2. CLASE DE ANÁLISIS FUNCIONAL (Lógica encapsulada)
# ==============================================================================
class AnalizadorFuncional:
    def __init__(self, df, config):
        self.df_original = df.copy()
        self.config = config
        self.target_col = config['VARIABLE_OBJETIVO']
        self.freq = config['FRECUENCIA_MUESTREO']
        self.fd_global = None
        self.labels_meses = None

    def _hex_to_rgba(self, hex_color, alpha):
        rgb = mcolors.to_rgb(hex_color)
        return f"rgba({rgb[0]*255},{rgb[1]*255},{rgb[2]*255},{alpha})"

    def limpiar_datos(self):
        """Aplica filtros de rango y convierte fechas."""
        print(f"🧹 Limpiando variable: {self.target_col}...")
        
        # Asegurar datetime
        self.df_original['datetime'] = pd.to_datetime(self.df_original['datetime'], errors='coerce')
        self.df_original = self.df_original.dropna(subset=['datetime'])

        # Filtrar valores extremos (Outliers técnicos)
        limites = self.config.get('LIMITES_LIMPIEZA')
        if limites:
            col_data = self.df_original[self.target_col]
            mask = (col_data < limites.get('min', -np.inf)) | (col_data > limites.get('max', np.inf))
            
            n_outliers = mask.sum()
            if n_outliers > 0:
                mediana = col_data.median()
                self.df_original.loc[mask, self.target_col] = mediana
                print(f"   -> Se corrigieron {n_outliers} valores fuera de rango (Reemplazados por mediana: {mediana:.2f})")

    def preparar_fda(self):
        """Transforma el DataFrame lineal a Objeto Funcional (FDataGrid)."""
        print("⚙️ Generando matriz funcional...")
        
        # 1. Filtro Temporal (Meses)
        df = self.df_original[self.df_original['datetime'].dt.month.isin(self.config['MESES'])].copy()
        
        if df.empty:
            raise ValueError("❌ No hay datos para los meses seleccionados.")

        # 2. Resampling (Estandarización de la grilla temporal)
        # Agrupamos por hora para tener una grilla uniforme (0h a 24h)
        df_resampled = (df.set_index('datetime')
                          .resample(self.freq)[self.target_col]
                          .mean()
                          .reset_index())

        # Crear ejes para pivotar
        df_resampled['fecha_solo'] = df_resampled['datetime'].dt.date
        
        # Calcular hora decimal (Eje X del gráfico funcional)
        # Ej: 14:30 -> 14.5
        df_resampled['hora_decimal'] = (df_resampled['datetime'].dt.hour + 
                                        df_resampled['datetime'].dt.minute / 60.0)

        # 3. Pivotar (Filas=Días, Columnas=Horas)
        matrix = df_resampled.pivot(index='fecha_solo', columns='hora_decimal', values=self.target_col)

        # 4. Interpolación (Manejo de Nulls)
        # Rellenar huecos internos y extremos
        matrix = matrix.interpolate(axis=1, limit_direction='both')
        matrix = matrix.bfill(axis=1).ffill(axis=1).dropna()

        print(f"   -> Dimensiones finales: {matrix.shape} (Días x Puntos)")

        if matrix.empty:
            raise ValueError("❌ La matriz funcional quedó vacía tras el procesamiento.")

        # 5. Crear Objeto scikit-fda
        self.fd_global = skfda.FDataGrid(
            data_matrix=matrix.values,
            grid_points=matrix.columns.values
        )
        
        # Guardar etiquetas de meses para el subplot
        fechas_index = pd.to_datetime(matrix.index)
        self.labels_meses = fechas_index.month.to_numpy()

    def _extraer_componentes_boxplot(self, bp_obj):
        """Helper robusto para extraer datos de numpy/tuples/fdatagrid."""
        res = {}
        
        def _to_numpy_flat(obj):
            if hasattr(obj, 'data_matrix'): return obj.data_matrix[..., 0].flatten()
            return np.array(obj).flatten()

        # Mediana
        res['mediana'] = _to_numpy_flat(bp_obj.median)

        # Banda Central (50%)
        central = bp_obj.central_envelope
        if hasattr(central, 'data_matrix'):
            res['banda_inf'] = central.data_matrix[0, ..., 0].flatten()
            res['banda_sup'] = central.data_matrix[1, ..., 0].flatten()
        else:
            # Fallback para versiones antiguas o tuplas
            arr = np.array(central)
            if arr.ndim > 1:
                res['banda_inf'] = arr[0].flatten()
                res['banda_sup'] = arr[1].flatten()
            else:
                 res['banda_inf'] = res['banda_sup'] = arr.flatten()

        # Bigotes (Non-outlying)
        whisker = bp_obj.non_outlying_envelope
        if hasattr(whisker, 'data_matrix'):
            res['bigote_inf'] = whisker.data_matrix[0, ..., 0].flatten()
            res['bigote_sup'] = whisker.data_matrix[1, ..., 0].flatten()
        else:
            arr = np.array(whisker)
            if arr.ndim > 1:
                res['bigote_inf'] = arr[0].flatten()
                res['bigote_sup'] = arr[1].flatten()
            else:
                 res['bigote_inf'] = res['bigote_sup'] = arr.flatten()
                 
        return res

    def graficar(self):
        """Genera el gráfico interactivo Plotly."""
        if self.fd_global is None:
            print("⚠️ Primero debes ejecutar 'preparar_fda()'")
            return

        meses_activos = [m for m in self.config['MESES'] if m in np.unique(self.labels_meses)]
        n_cols = len(meses_activos)
        
        if n_cols == 0:
            print("⚠️ No hay datos coincidentes para graficar.")
            return

        titulos = [f"<b>{self.config['NOMBRES_MESES'].get(m, m)}</b> (n={np.sum(self.labels_meses==m)})" 
                   for m in meses_activos]

        fig = make_subplots(
            rows=1, cols=n_cols,
            subplot_titles=titulos,
            shared_yaxes=True,
            horizontal_spacing=0.03
        )

        x_vals = self.fd_global.grid_points[0]
        colores = self.config['COLORES']
        color_relleno = self._hex_to_rgba(colores['banda'], 0.4)

        print("📊 Generando gráfico...")

        for i, mes in enumerate(meses_activos):
            col_idx = i + 1
            
            # Filtro por mes
            fd_mes = self.fd_global[self.labels_meses == mes]
            
            if fd_mes.n_samples < 5:
                print(f"   -> Saltando mes {mes} (datos insuficientes < 5)")
                continue

            # Cálculo Estadístico
            try:
                bp = Boxplot(fd_mes, depth_method=ModifiedBandDepth())
                datos = self._extraer_componentes_boxplot(bp)
            except Exception as e:
                print(f"   -> Error en mes {mes}: {e}")
                continue

            # --- PLOTTING ---
            mostrar_leyenda = (i == 0)

            # 1. Banda Central (Fill)
            fig.add_trace(go.Scatter(
                x=x_vals, y=datos['banda_inf'], mode='lines', line=dict(width=0), 
                showlegend=False, hoverinfo='skip'
            ), row=1, col=col_idx)
            
            fig.add_trace(go.Scatter(
                x=x_vals, y=datos['banda_sup'], mode='lines', line=dict(width=0),
                fill='tonexty', fillcolor=color_relleno,
                name='Rango 50%', showlegend=mostrar_leyenda
            ), row=1, col=col_idx)

            # 2. Bordes y Bigotes
            estilo_linea = dict(color=colores['banda'], width=1)
            estilo_bigote = dict(color=colores['banda'], width=1.5, dash='dash')

            # Bordes banda
            fig.add_trace(go.Scatter(x=x_vals, y=datos['banda_sup'], mode='lines', line=estilo_linea, showlegend=False), row=1, col=col_idx)
            fig.add_trace(go.Scatter(x=x_vals, y=datos['banda_inf'], mode='lines', line=estilo_linea, showlegend=False), row=1, col=col_idx)

            # Bigotes
            fig.add_trace(go.Scatter(
                x=x_vals, y=datos['bigote_sup'], mode='lines', line=estilo_bigote,
                name='Límites No-Outlier', showlegend=mostrar_leyenda
            ), row=1, col=col_idx)
            fig.add_trace(go.Scatter(x=x_vals, y=datos['bigote_inf'], mode='lines', line=estilo_bigote, showlegend=False), row=1, col=col_idx)

            # 3. Mediana
            fig.add_trace(go.Scatter(
                x=x_vals, y=datos['mediana'],
                mode='lines', line=dict(color=colores['mediana'], width=3),
                name='Mediana', showlegend=mostrar_leyenda
            ), row=1, col=col_idx)

        # Configuración final del layout
        axis_style = dict(
            showline=True, linewidth=1, linecolor=colores['marco'], mirror=True,
            showgrid=True, gridcolor=colores['grid'], gridwidth=1, zeroline=False
        )
        
        fig.update_layout(
            title_text=f"<b>Variabilidad Funcional: {self.target_col.upper()}</b>",
            title_x=0.5,
            template="plotly_white",
            height=500,
            margin=dict(l=50, r=50, t=80, b=50),
            legend=dict(orientation="h", yanchor="bottom", y=1.05, xanchor="center", x=0.5)
        )
        
        fig.update_xaxes(title_text="Hora del Día", range=[0, 24], **axis_style)
        fig.update_yaxes(**axis_style)
        
        return fig




In [54]:

# ==============================================================================
# 3. EJECUCIÓN (Interfaz simple)
# ==============================================================================

   
config_usuario = CONFIG.copy()
config_usuario['VARIABLE_OBJETIVO'] = 'ph_chiller'  # Cambiar a 'cloro_chiller' u 'orp_chiller'
config_usuario['FRECUENCIA_MUESTREO'] = '60min'     # Puedes cambiar a '15min' o '1h'
config_usuario['LIMITES_LIMPIEZA'] = {'min': 0, 'max': 14} # Filtro lógico para pH


analisis = AnalizadorFuncional(sensores, config_usuario)
analisis.limpiar_datos()
analisis.preparar_fda()
figura = analisis.graficar()
figura.show()


🧹 Limpiando variable: ph_chiller...
⚙️ Generando matriz funcional...
   -> Dimensiones finales: (53, 24) (Días x Puntos)
📊 Generando gráfico...


In [55]:
config_usuario = CONFIG.copy()
config_usuario['VARIABLE_OBJETIVO'] = 'orp_chiller'  # Cambiar a 'cloro_chiller' u 'orp_chiller'
config_usuario['FRECUENCIA_MUESTREO'] = '60min'     # Puedes cambiar a '15min' o '1h'
config_usuario['LIMITES_LIMPIEZA'] = {'min': 200, 'max': 900} # Filtro lógico para pH


analisis = AnalizadorFuncional(sensores, config_usuario)
analisis.limpiar_datos()
analisis.preparar_fda()
figura = analisis.graficar()
figura.show()

🧹 Limpiando variable: orp_chiller...
   -> Se corrigieron 3 valores fuera de rango (Reemplazados por mediana: 496.00)
⚙️ Generando matriz funcional...
   -> Dimensiones finales: (53, 24) (Días x Puntos)
📊 Generando gráfico...


In [56]:
config_usuario = CONFIG.copy()
config_usuario['VARIABLE_OBJETIVO'] = 'cloro_chiller'  # Cambiar a 'cloro_chiller' u 'orp_chiller'
config_usuario['FRECUENCIA_MUESTREO'] = '60min'     # Puedes cambiar a '15min' o '1h'
config_usuario['LIMITES_LIMPIEZA'] = {'min': 2, 'max': 6} # Filtro lógico para pH


analisis = AnalizadorFuncional(sensores, config_usuario)
analisis.limpiar_datos()
analisis.preparar_fda()
figura = analisis.graficar()
figura.show()

🧹 Limpiando variable: cloro_chiller...
⚙️ Generando matriz funcional...
   -> Dimensiones finales: (53, 24) (Días x Puntos)
📊 Generando gráfico...


In [57]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.colors as mcolors
import skfda
from skfda.exploratory.visualization import Boxplot
from skfda.exploratory.depth import ModifiedBandDepth


# ==============================================================================
# 2. CLASE DE ANÁLISIS FUNCIONAL
# ==============================================================================
class AnalizadorFuncional:
    def __init__(self, df, config):
        self.df_original = df.copy()
        self.config = config
        self.target_col = config['VARIABLE_OBJETIVO']
        self.freq = config['FRECUENCIA_MUESTREO']
        self.fd_global = None
        self.labels_meses = None

    def _hex_to_rgba(self, hex_color, alpha):
        rgb = mcolors.to_rgb(hex_color)
        return f"rgba({rgb[0]*255},{rgb[1]*255},{rgb[2]*255},{alpha})"

    def limpiar_datos(self):
        """Aplica filtros de rango y convierte fechas."""
        print(f"🧹 Limpiando variable: {self.target_col}...")
        
        self.df_original['datetime'] = pd.to_datetime(self.df_original['datetime'], errors='coerce')
        self.df_original = self.df_original.dropna(subset=['datetime'])

        limites = self.config.get('LIMITES_LIMPIEZA')
        if limites:
            col_data = self.df_original[self.target_col]
            mask = (col_data < limites.get('min', -np.inf)) | (col_data > limites.get('max', np.inf))
            
            n_outliers = mask.sum()
            if n_outliers > 0:
                mediana = col_data.median()
                self.df_original.loc[mask, self.target_col] = mediana
                print(f"   -> Se corrigieron {n_outliers} valores fuera de rango.")

    def preparar_fda(self):
        """Transforma el DataFrame lineal a Objeto Funcional (FDataGrid)."""
        print("⚙️ Generando matriz funcional...")
        
        df = self.df_original[self.df_original['datetime'].dt.month.isin(self.config['MESES'])].copy()
        
        if df.empty:
            raise ValueError("❌ No hay datos para los meses seleccionados.")

        df_resampled = (df.set_index('datetime')
                          .resample(self.freq)[self.target_col]
                          .mean()
                          .reset_index())

        df_resampled['fecha_solo'] = df_resampled['datetime'].dt.date
        df_resampled['hora_decimal'] = (df_resampled['datetime'].dt.hour + 
                                        df_resampled['datetime'].dt.minute / 60.0)

        matrix = df_resampled.pivot(index='fecha_solo', columns='hora_decimal', values=self.target_col)
        matrix = matrix.interpolate(axis=1, limit_direction='both')
        matrix = matrix.bfill(axis=1).ffill(axis=1).dropna()

        print(f"   -> Dimensiones finales: {matrix.shape} (Días x Puntos)")

        if matrix.empty:
            raise ValueError("❌ La matriz funcional quedó vacía tras el procesamiento.")

        self.fd_global = skfda.FDataGrid(
            data_matrix=matrix.values,
            grid_points=matrix.columns.values
        )
        
        fechas_index = pd.to_datetime(matrix.index)
        self.labels_meses = fechas_index.month.to_numpy()

    def _extraer_componentes_boxplot(self, bp_obj):
        """Helper robusto para extraer datos de numpy/tuples/fdatagrid."""
        res = {}
        
        def _to_numpy_flat(obj):
            if hasattr(obj, 'data_matrix'): return obj.data_matrix[..., 0].flatten()
            return np.array(obj).flatten()

        res['mediana'] = _to_numpy_flat(bp_obj.median)

        central = bp_obj.central_envelope
        if hasattr(central, 'data_matrix'):
            res['banda_inf'] = central.data_matrix[0, ..., 0].flatten()
            res['banda_sup'] = central.data_matrix[1, ..., 0].flatten()
        else:
            arr = np.array(central)
            if arr.ndim > 1:
                res['banda_inf'] = arr[0].flatten()
                res['banda_sup'] = arr[1].flatten()
            else:
                 res['banda_inf'] = res['banda_sup'] = arr.flatten()

        whisker = bp_obj.non_outlying_envelope
        if hasattr(whisker, 'data_matrix'):
            res['bigote_inf'] = whisker.data_matrix[0, ..., 0].flatten()
            res['bigote_sup'] = whisker.data_matrix[1, ..., 0].flatten()
        else:
            arr = np.array(whisker)
            if arr.ndim > 1:
                res['bigote_inf'] = arr[0].flatten()
                res['bigote_sup'] = arr[1].flatten()
            else:
                 res['bigote_inf'] = res['bigote_sup'] = arr.flatten()
                 
        return res

    # --------------------------------------------------------------------------
    # MÉTODO 1: BOXPLOT (El que ya tenías)
    # --------------------------------------------------------------------------
    def graficar(self):
        """Genera el gráfico interactivo Plotly (Boxplot Funcional)."""
        if self.fd_global is None:
            print("⚠️ Primero debes ejecutar 'preparar_fda()'")
            return

        meses_activos = [m for m in self.config['MESES'] if m in np.unique(self.labels_meses)]
        n_cols = len(meses_activos)
        
        if n_cols == 0: return

        titulos = [f"<b>{self.config['NOMBRES_MESES'].get(m, m)}</b> (n={np.sum(self.labels_meses==m)})" 
                   for m in meses_activos]

        fig = make_subplots(
            rows=1, cols=n_cols,
            subplot_titles=titulos,
            shared_yaxes=True,
            horizontal_spacing=0.03
        )

        x_vals = self.fd_global.grid_points[0]
        colores = self.config['COLORES']
        color_relleno = self._hex_to_rgba(colores['banda'], 0.4)

        print("📊 Generando gráfico de Boxplots...")

        for i, mes in enumerate(meses_activos):
            col_idx = i + 1
            fd_mes = self.fd_global[self.labels_meses == mes]
            
            if fd_mes.n_samples < 5: continue

            try:
                bp = Boxplot(fd_mes, depth_method=ModifiedBandDepth())
                datos = self._extraer_componentes_boxplot(bp)
            except Exception as e:
                print(f"   -> Error en mes {mes}: {e}")
                continue

            mostrar_leyenda = (i == 0)

            # Banda Central
            fig.add_trace(go.Scatter(x=x_vals, y=datos['banda_inf'], mode='lines', line=dict(width=0), showlegend=False, hoverinfo='skip'), row=1, col=col_idx)
            fig.add_trace(go.Scatter(x=x_vals, y=datos['banda_sup'], mode='lines', line=dict(width=0), fill='tonexty', fillcolor=color_relleno, name='Rango 50%', showlegend=mostrar_leyenda), row=1, col=col_idx)

            # Bordes
            estilo_linea = dict(color=colores['banda'], width=1)
            fig.add_trace(go.Scatter(x=x_vals, y=datos['banda_sup'], mode='lines', line=estilo_linea, showlegend=False), row=1, col=col_idx)
            fig.add_trace(go.Scatter(x=x_vals, y=datos['banda_inf'], mode='lines', line=estilo_linea, showlegend=False), row=1, col=col_idx)

            # Bigotes
            estilo_bigote = dict(color=colores['banda'], width=1.5, dash='dash')
            fig.add_trace(go.Scatter(x=x_vals, y=datos['bigote_sup'], mode='lines', line=estilo_bigote, name='Límites No-Outlier', showlegend=mostrar_leyenda), row=1, col=col_idx)
            fig.add_trace(go.Scatter(x=x_vals, y=datos['bigote_inf'], mode='lines', line=estilo_bigote, showlegend=False), row=1, col=col_idx)

            # Mediana
            fig.add_trace(go.Scatter(x=x_vals, y=datos['mediana'], mode='lines', line=dict(color=colores['mediana'], width=3), name='Mediana', showlegend=mostrar_leyenda), row=1, col=col_idx)

        axis_style = dict(showline=True, linewidth=1, linecolor=colores['marco'], mirror=True, showgrid=True, gridcolor=colores['grid'])
        fig.update_layout(title_text=f"<b>Variabilidad Funcional: {self.target_col.upper()}</b>", title_x=0.5, template="plotly_white", height=500, legend=dict(orientation="h", yanchor="bottom", y=1.05, xanchor="center", x=0.5))
        fig.update_xaxes(title_text="Hora del Día", range=[0, 24], **axis_style)
        fig.update_yaxes(**axis_style)
        
        return fig

    # --------------------------------------------------------------------------
    # MÉTODO 2: VELOCIDAD (NUEVO - INCLUIDO AQUÍ)
    # --------------------------------------------------------------------------
    def graficar_velocidad(self):
        """
        Calcula la derivada (velocidad de cambio) y compara el promedio mensual.
        """
        if self.fd_global is None:
            print("⚠️ Primero debes ejecutar 'preparar_fda()'")
            return None
        
        print("📈 Calculando Velocidad de Cambio (Derivadas)...")
        
        # 1. Cálculo matemático de la derivada (FDA)
        fd_deriv = self.fd_global.derivative()
        x_vals = self.fd_global.grid_points[0]
        
        fig = go.Figure()
        
        # Línea de referencia (0)
        fig.add_hline(y=0, line_dash="dot", line_color="black", opacity=0.5)

        meses_activos = [m for m in self.config['MESES'] if m in np.unique(self.labels_meses)]
        
        # Colores por defecto si no existen en config
        colores_meses = self.config.get('COLORES_MESES', {})

        for mes in meses_activos:
            nombre_mes = self.config['NOMBRES_MESES'].get(mes, str(mes))
            color_hex = colores_meses.get(mes, '#778899') # Gris por defecto
            
            # Filtramos los datos (derivadas) de este mes
            fd_mes_deriv = fd_deriv[self.labels_meses == mes]
            
            if fd_mes_deriv.n_samples == 0: continue
            
            # Promedio de la velocidad del mes
            mean_obj = fd_mes_deriv.mean()
            
            # -- Extracción Segura de la media (similar al helper) --
            if hasattr(mean_obj, 'data_matrix'):
                mean_velocity = mean_obj.data_matrix[..., 0].flatten()
            else:
                mean_velocity = np.array(mean_obj).flatten()
            
            # Color con transparencia para el relleno
            color_fill = self._hex_to_rgba(color_hex, 0.1) 
            
            fig.add_trace(go.Scatter(
                x=x_vals,
                y=mean_velocity,
                mode='lines',
                name=nombre_mes,
                line=dict(color=color_hex, width=3),
                fill='tozeroy',      # Relleno hacia el cero
                fillcolor=color_fill
            ))

        # Estética
        colores_estilo = self.config['COLORES']
        fig.update_layout(
            title_text=f"<b>Velocidad de Cambio Promedio (Volatilidad): {self.target_col.upper()}</b>",
            title_x=0.5,
            xaxis_title="Hora del Día",
            yaxis_title=f"Velocidad ({self.target_col}/h)",
            template="plotly_white",
            height=500,
            hovermode="x unified", # Muestra todos los valores al pasar el mouse por una hora
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="center", x=0.5)
        )
        
        axis_style = dict(showline=True, linewidth=1, linecolor=colores_estilo['marco'], mirror=True, showgrid=True, gridcolor=colores_estilo['grid'])
        fig.update_xaxes(range=[0, 24], **axis_style)
        fig.update_yaxes(**axis_style)
        
        return fig

In [58]:
# ==============================================================================
# 1. CONFIGURACIÓN GENERAL
# ==============================================================================
CONFIG = {
    # Columnas disponibles: 'cloro_chiller', 'ph_chiller', 'orp_chiller'
    'VARIABLE_OBJETIVO': 'ph_chiller',  
    'FRECUENCIA_MUESTREO': '1h',
    
    # Filtro de meses
    'MESES': [10, 11, 12],
    'NOMBRES_MESES': {10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'},
    
    # Colores específicos para el gráfico de Velocidad (Diferenciación visual)
    'COLORES_MESES': {
        10: '#1C8074', # Verde (Oct)
        11: '#666666', # Azul (Nov)
        12: '#E4572E', # Rojo (Dic)
        1:  '#29B6F6'  # Amarillo (Ene - por si acaso)
    },
    # Limpieza
    'LIMITES_LIMPIEZA': {'min': 0, 'max': 8},

    # Estética General (Boxplot)
    'COLORES': {
        'mediana': "#1C8074",       # Negro
        'banda': "#666666",         # Gris Azulado
        'marco': "#000000",
        'grid': "#000000"
    }
}


In [59]:

analisis = AnalizadorFuncional(sensores, CONFIG)
analisis.limpiar_datos()
analisis.preparar_fda()
fig1 = analisis.graficar()
s3.save_plotly_html(fig1, 'boxplot_funcional_ph_chiller.html')
fig1.show()
fig2 = analisis.graficar_velocidad()
s3.save_plotly_html(fig2, 'velocidad_funcional_ph_chiller.html')
fig2.show()


🧹 Limpiando variable: ph_chiller...
⚙️ Generando matriz funcional...
   -> Dimensiones finales: (53, 24) (Días x Puntos)
📊 Generando gráfico de Boxplots...


📈 Calculando Velocidad de Cambio (Derivadas)...


In [60]:
CONFIG["VARIABLE_OBJETIVO"] = 'orp_chiller' 
CONFIG['LIMITES_LIMPIEZA'] = {'min': 200, 'max': 900}
analisis = AnalizadorFuncional(sensores, CONFIG)
analisis.limpiar_datos()
analisis.preparar_fda()
fig1 = analisis.graficar()
s3.save_plotly_html(fig1, 'boxplot_funcional_orp_chiller.html')
fig1.show()
fig2 = analisis.graficar_velocidad()
s3.save_plotly_html(fig2, 'velocidad_funcional_orp_chiller.html')
fig2.show()

🧹 Limpiando variable: orp_chiller...
   -> Se corrigieron 3 valores fuera de rango.
⚙️ Generando matriz funcional...
   -> Dimensiones finales: (53, 24) (Días x Puntos)
📊 Generando gráfico de Boxplots...


📈 Calculando Velocidad de Cambio (Derivadas)...


In [61]:
CONFIG["VARIABLE_OBJETIVO"] = 'cloro_chiller' 
CONFIG['LIMITES_LIMPIEZA'] = {'min': 2, 'max': 6}
analisis = AnalizadorFuncional(sensores, CONFIG)
analisis.limpiar_datos()
analisis.preparar_fda()
fig1 = analisis.graficar()
s3.save_plotly_html(fig1, "boxplot_funcional_cloro_chiller.html")
fig1.show()
fig2 = analisis.graficar_velocidad()
s3.save_plotly_html(fig2, "velocidad_funcional_cloro_chiller.html")
fig2.show()

🧹 Limpiando variable: cloro_chiller...
⚙️ Generando matriz funcional...
   -> Dimensiones finales: (53, 24) (Días x Puntos)
📊 Generando gráfico de Boxplots...


📈 Calculando Velocidad de Cambio (Derivadas)...
